In [371]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import GPy

from Booster_single_obj_new import Booster

In [372]:
pts = 20
X = np.random.uniform(0, 1, (pts, 4))
y = np.random.uniform(0, 0, (pts, 1))
Cons = np.random.uniform(0, 0, (9, pts))
for i in range(pts):
    y[i],Cons[:,i]=Booster(X[i,:])
    y[i] = -y[i]
#y = np.array([objective(x1,x2) for (x1,x2) in zip(X[:,0], X[:,1])])[:,None]
k = GPy.kern.RBF(4)
m = GPy.models.GPRegression(X, y, k)
m.optimize()
xi = 0.01


In [373]:
mean_cons = np.zeros((9))
std_cons = np.zeros((9))
for i in range(9):
    mean_cons[i] = np.mean(Cons[:,i])
    std_cons[i] = np.std(Cons[:, i])
ratio = mean_cons/std_cons
cdf = norm.cdf(ratio)
term = np.prod(cdf)

In [374]:
def EI(mu, best, std):
    return (mu - best)*norm.cdf((mu - best - xi) / (std))+std*norm.pdf((mu - best) / (std))

def PI(mu, best, std):
    return norm.cdf((mu - best - xi) / (std+1E-9))

def UCB(mu, best, std):
    return mu + xi*std

def Direct(mu, best, std):
    return ((mu - best)*norm.cdf((mu - best - xi) / (std))+std*norm.pdf((mu - best) / (std)))*term

def POF(mu, best, std):
    return (mu - best)*norm.cdf((mu - best - xi)/(std))+std*norm.pdf((mu-best)/(std))*term

In [375]:
# perform the optimization process
# select the next point to sample
iter = 60
for i in range(iter):
    Xsamples = np.random.uniform(0, 1, (pts, 4))
    # calculate the acquisition function for each sample

    yhat, _ = m.predict(X, full_cov=False)
    best = np.max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = m.predict(Xsamples, full_cov=False)
    # calculate the probability of improvement
    Acq = EI(mu, best, std)
    #Acq = PI(mu, best, std)
    #Acq = UCB(mu, best, std)
    #Acq = Direct(mu, best, std)
    #Acq = POF(mu, best, std)
    

    # locate the index of the largest scores
    ix = np.argmax(Acq)
    x = Xsamples[ix, :][:,None].T
    # sample the point
    x1 = x[0,0]
    x2 = x[0,1]
    x3 = x[0,2]
    x4 = x[0,3]

    for i in range(1):
        actual,Cons[:,i]=Booster(x[i,:])

        # summarize the finding
    #est, _ = m.predict([[x]], full_cov=False)
    #print("est: ",est)
    #print('>x=%.3f, f()=%3f, actual=%.3f' % (x, est, actual))
        # add the data to the dataset

    X = np.concatenate((X, x), axis=0)
    y = np.vstack((y, [[actual]]))
        # update the model
    m = GPy.models.GPRegression(X, y, k)

# best result
ix = np.argmax(y)
print('Best Result:%.0f & %.3f & %.3f & %.3f & %.3f & %.3f \\\ \hline' % (iter, X[ix,0], X[ix,1], X[ix,2], X[ix,3], y[ix]))

Best Result:60 & 0.189 & 0.362 & 0.348 & 0.935 & 5.145 \\ \hline
